In [18]:
from sklearn import ensemble, model_selection, metrics, datasets, tree

import numpy as np
import pandas as pd

In [8]:
from sklearn.datasets import load_digits
digits = load_digits()

In [14]:
X = digits.data
Y = digits.target

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score

In [20]:

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 1)
tree.fit(X, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [26]:
scoring = model_selection.cross_val_score(tree, X, Y, cv = 10)
scoring.mean()

0.8308624439503834

Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifie

In [28]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator = tree, n_estimators=100, random_state = 1).fit(X, Y)

In [31]:
scoring = model_selection.cross_val_score(clf, X, Y, cv = 10)
scoring.mean()

0.9271102572603436

Изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, 
а на sqrt{d} случайных признаков.

In [46]:
import math
n = int(math.sqrt(X.shape[1]))
clf_n = BaggingClassifier(base_estimator = tree, n_estimators=100,
                              max_features = n , random_state = 1).fit(X, Y)

In [47]:
scoring = model_selection.cross_val_score(clf_n, X, Y, cv = 10)
scoring.mean()

0.933810128907554

Давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. 
Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier

In [52]:
tree_rand = DecisionTreeClassifier(max_features = n ,random_state = 1)
tree_rand.fit(X, Y)
clf_rand = BaggingClassifier(base_estimator = tree_rand, n_estimators=100, random_state = 1).fit(X, Y)

In [53]:
scoring = model_selection.cross_val_score(clf_rand, X, Y, cv = 10)
scoring.mean()

0.9500349636021452

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble

In [88]:
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 15,max_depth=6,max_features =  60,random_state = 1)
scoring = model_selection.cross_val_score(rf_classifier, X, Y, cv = 10)
scoring.mean()

0.8592212455152002